In [ ]:
%pwd

'e:\\Bappy\\Coding\\Youtube\\Generative AI\\Langchain-Projects\\End-to-end-Medical-Chatbot-Generative-AI\\research'

# Loading the current directory to Extract the PDF File

In [ ]:
import os
os.chdir("../")

In [ ]:
%pwd

'e:\\Bappy\\Coding\\Youtube\\Generative AI\\Langchain-Projects\\End-to-end-Medical-Chatbot-Generative-AI'

# Loading the Document Splitter, Loader Classes from Langchain

In [ ]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Defining a Custom Load PDF function to Ingest the Data

In [ ]:

def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)

    documents=loader.load()

    return documents


# Calling the Load PDF Function

In [ ]:
extracted_data=load_pdf_file(data='Data/')

# Text Splitting
## Defing Chunk Size as 500 and Overlap Size as 20

In [ ]:

def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [ ]:
text_chunks=text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 7020


# Downloading an Embedding Model from HuggingFcae

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings

c:\Anaconda3\envs\medibot\lib\site-packages\pydantic\_internal\_fields.py:132: UserWarning: Field "model_name" in HuggingFaceInferenceAPIEmbeddings has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


# We will use sentemce transformers as our embedding model

In [ ]:

def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings


In [ ]:
embeddings = download_hugging_face_embeddings()

C:\Users\Bappy Ahmed\AppData\Local\Temp\ipykernel_21168\1196424635.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
c:\Anaconda3\envs\medibot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Anaconda3\envs\medibot\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be

In [ ]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [ ]:
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')
COHERE_API_KEY = os.getenv('COHERE_KEY')
GROQ_API_KEY = os.getenv('GROQ_API_KEY')

In [ ]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key=PINECONE_API_KEY)

# this is the database name
index_name = "tedtalks"


pc.create_index(
    name=index_name, # database name
    dimension=384,
    metric="cosine" , # we define the similarity metric in this case it is cosine similarity
    spec=ServerlessSpec(
        cloud="aws", #  we host pinecone on AWS
        region="us-east-1"
    )
)

We set up the environment variables they are API keys which are initilized in your working directory. ALL you have to do is make '.env' file. It has no other name. You have to define the api in the format as below

PINECONE_API = 'Write your API'

In [ ]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

We load the text chunk and embeddings to th e pinecone database

In [ ]:

from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings,
)

# Loading all the documents from the Vector Store
Now we import the embeddings from the remote vector store

In [ ]:

from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)